In [1]:
import cv2
import json
import numpy as np
from pycocotools import mask as mask_util
from typing import Dict, Any

# ===============================================
# 📌 사용자 설정 (User Configuration)
# ===============================================

# 1. JSON 파일 경로를 설정하세요. (RLE 마스크 데이터 포함)
JSON_FILE_PATH = '0188f9be-7186-4cb7-9a97-5c30ac6eb111_masks.json' 

# 2. 원본 비디오 파일 경로를 설정하세요.
VIDEO_FILE_PATH = '1_4_8mp4.mp4' 

# 3. 결과물이 저장될 비디오 파일 경로를 설정하세요.
OUTPUT_VIDEO_PATH = 'masked_output.mp4'

# 4. 마스크를 적용할 색상 (BGR 포맷: Blue, Green, Red)
# (예: (0, 0, 255)는 빨간색, (0, 255, 0)는 초록색)
MASK_COLOR = (0, 0, 255) 

# 5. 마스크 투명도 (0.0: 완전 투명, 1.0: 완전 불투명)
ALPHA = 0.4 

# ===============================================
# ⚙️ 함수 정의 (Function Definitions)
# ===============================================

def rle_to_binary_mask(rle: Dict[str, Any]) -> np.ndarray:
    """
    RLE(Run-Length Encoding) 데이터를 바이너리 마스크(NumPy 배열)로 변환합니다.
    pycocotools 라이브러리를 사용하여 디코딩합니다.
    """
    if 'counts' in rle and 'size' in rle:
        # pycocotools의 RLE 포맷에 맞게 딕셔너리 구조를 맞춥니다.
        rle_obj = {
            'counts': rle['counts'],
            'size': rle['size']
        }
        # RLE 디코딩을 통해 바이너리 마스크를 얻습니다.
        # 결과는 (Height, Width) 형태의 uint8 배열입니다.
        binary_mask = mask_util.decode(rle_obj)
        return binary_mask
    else:
        # 올바른 RLE 포맷이 아닐 경우 빈 배열 반환
        return np.array([])


def apply_mask_to_frame(frame: np.ndarray, mask_data: Dict[str, Any]) -> np.ndarray:
    """
    단일 프레임에 JSON 마스크 데이터를 적용합니다.
    """
    frame_height, frame_width, _ = frame.shape
    output_frame = frame.copy()

    # 'results' 키 아래에 있는 모든 객체 마스크를 순회 (0, 1, 2...)
    results = mask_data.get('results', {})
    
    # 모든 마스크를 누적할 임시 배열
    aggregated_mask_colored = np.zeros_like(frame, dtype=np.uint8)

    # 객체 ID별로 마스크를 순회
    for obj_id, obj_data in results.items():
        mask_rle = obj_data.get('mask', {})
        
        # RLE를 바이너리 마스크로 변환
        mask_array = rle_to_binary_mask(mask_rle)
        
        if mask_array.size > 0 and mask_array.shape == (frame_height, frame_width):
            # 2D 마스크 배열에서 마스크가 씌워질 영역을 찾고 색상을 적용
            mask_colored = np.zeros_like(frame, dtype=np.uint8)
            mask_colored[mask_array > 0] = MASK_COLOR
            
            # 모든 객체의 마스크 색상을 누적하여 오버랩을 처리
            # 이 방식은 겹치는 부분의 색상을 마지막으로 처리된 객체의 색상으로 덮어씁니다.
            aggregated_mask_colored[mask_array > 0] = MASK_COLOR
            
    # 누적된 마스크 색상과 원본 프레임을 투명도를 적용하여 합성
    if np.any(aggregated_mask_colored):
        # cv2.addWeighted(src1, alpha, src2, beta, gamma)
        # output = frame * (1.0 - ALPHA) + aggregated_mask_colored * ALPHA
        output_frame = cv2.addWeighted(frame, 1.0 - ALPHA, aggregated_mask_colored, ALPHA, 0)
    
    return output_frame


def process_video_with_masks(json_path: str, video_path: str, output_path: str):
    """
    JSON 파일의 마스크 데이터를 비디오 프레임에 적용하고 새 비디오로 저장합니다.
    """
    # 1. JSON 데이터 로드
    try:
        with open(json_path, 'r') as f:
            full_data = json.load(f)
        # 'frames' 리스트를 프레임 인덱스를 키로 하는 딕셔너리로 변환하여 빠른 접근을 가능하게 합니다.
        frame_data_map = {item.get('frame_index'): item for item in full_data.get('root', {}).get('frames', [])}
    except FileNotFoundError:
        print(f"❌ Error: JSON file not found at {json_path}")
        return
    except json.JSONDecodeError:
        print(f"❌ Error: Failed to decode JSON from {json_path}")
        return
    except Exception as e:
        print(f"❌ An unexpected error occurred during JSON loading: {e}")
        return

    # 2. 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Error: Could not open video file {video_path}")
        return

    # 3. 비디오 속성 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if frame_width == 0 or frame_height == 0:
        print("❌ Error: Could not retrieve valid video dimensions (Width/Height is 0).")
        cap.release()
        return

    # 4. 비디오 쓰기 객체 생성
    # 'mp4v' 코덱은 가장 일반적인 MP4 코덱 중 하나입니다.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    print(f"🎬 Start processing video: {video_path}")
    print(f"  Total frames: {total_frames}, FPS: {fps:.2f}, Resolution: {frame_width}x{frame_height}")

    current_frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # 해당 프레임에 마스크 데이터가 있는지 확인
        mask_data = frame_data_map.get(current_frame_index)

        if mask_data:
            # 마스크 데이터를 프레임에 적용
            output_frame = apply_mask_to_frame(frame, mask_data)
        else:
            # 마스크 데이터가 없으면 원본 프레임 그대로 사용
            output_frame = frame

        # 새 프레임을 동영상에 기록
        out.write(output_frame)
        current_frame_index += 1

        # 진행 상황 출력 (100 프레임마다)
        if current_frame_index % 100 == 0:
            print(f"  Processed {current_frame_index}/{total_frames} frames...")

    # 5. 리소스 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"✅ Video processing complete.")
    print(f"    Output saved to: {OUTPUT_VIDEO_PATH}")

# ===============================================
# 🚀 메인 실행 (Main Execution)
# ===============================================
    
process_video_with_masks(JSON_FILE_PATH, VIDEO_FILE_PATH, OUTPUT_VIDEO_PATH)

ModuleNotFoundError: No module named 'cv2'